# Homework 2

## Interior point method implementation

In [186]:
import numpy as np

### 3)

In [187]:
# Compute 1 step of the iteration
def interior_point_method_step(A, b, c, x, y, s, mew):

    m = x.shape[0]

    # the mew update 
    mew_new = (1 - 1/(6 * np.sqrt(m))) * mew

    # Prepare the matrices for the Newton method
    S_inv = np.linalg.inv(np.diag(s))
    X = np.diag(x)
    e = np.ones_like(x)
     
    # Solve for update 
    k = np.linalg.inv(A @ S_inv @ X @ A.T) @ (b - mew_new * A @ S_inv @e)
    f = -A.T @ k
    h = -X @ S_inv @ f + mew_new * S_inv @ e - x

    # Update
    x_new = x + h
    y_new = y + k
    s_new = s + f

    return x_new, y_new, s_new, mew_new



### 6) 

In [188]:
A = np.array([
    [3, 3, 3, 0, 0],
    [3, 1, 0, 1, 0],
    [1, 4, 0, 0, 1]
], dtype=float)

b = np.array([4, 3, 4], dtype=float)

c = np.array([-3, -4, 0, 0, 0], dtype=float)
x = np.array([2/5, 8/15, 2/5, 19/15, 22/15], dtype=float)
y = np.array([-4/5, -4/5, -2/3], dtype=float)
s = np.array([37/15, 28/15, 12/5, 4/5, 2/3], dtype=float)
mew = 1

# Checks
print("Ax = b ?", np.allclose(A @ x, b))
print("A^T y + s = c ?", np.allclose(A.T @ y + s, c))
print("x > 0 ?", np.all(x > 0))
print("s > 0 ?", np.all(s > 0))

Ax = b ? True
A^T y + s = c ? True
x > 0 ? True
s > 0 ? True


### 7)

In [189]:
# mew = avg
print(np.sum((x * s/np.mean(x * s) - 1)**2))

# mew = 1
np.sum((x * s/1 - 1)**2)

0.0016232448664881226


np.float64(0.002469135802469144)

### 8)

In [190]:
# Iterating until convergence
epsilon = 1e-8
i = 0

while True:
    x_new, y_new, s, mew = interior_point_method_step(A, b, c, x, y, s, mew)
    
    if np.linalg.norm(x_new - x) < epsilon and np.linalg.norm(y_new - y) < epsilon:
        break

    x = x_new
    y= y_new
    i+=1


print("Optimal x:", x)
print(f"y: {y}")
print(f"s: {s}")
print(f"mew: {mew}")
print(f"it took {i} iterations")

Optimal x: [4.44444464e-01 8.88888854e-01 1.48994134e-08 7.77777753e-01
 1.19195297e-07]
y: [-8.88888861e-01 -5.10837032e-08 -3.33333352e-01]
s: [8.27332543e-08 4.13666304e-08 2.66666659e+00 4.72761487e-08
 3.33333351e-01]
mew: 3.677033676890448e-08
it took 220 iterations


### 9)

In [230]:
# Compute 1 step of the iteration
def interior_point_method_step_impr(A, b, c, x, y, s, mew, sigma = 0.5):

    m = x.shape[0]

    # the mew update 
    mew_new = (x @ s) / m       

    # Reduce mew as much as possible while staying in central path neighborhood
    mew_new = sigma * mew_new

    # mew_new = (1 - 1/(6 * np.sqrt(m))) * mew

    # Prepare the matrices for the Newton method
    S_inv = np.linalg.inv(np.diag(s))
    X = np.diag(x)
    e = np.ones_like(x)
     
    # Solve for update 
    k = np.linalg.inv(A @ S_inv @ X @ A.T) @ (b - mew_new * A @ S_inv @e)
    f = -A.T @ k
    h = -X @ S_inv @ f + mew_new * S_inv @ e - x

    # Update
    x_new = x + h
    y_new = y + k
    s_new = s + f

    assert np.all(x_new > 0), "x went non-positive!"
    assert np.all(s_new > 0), "s went non-positive!"

    central_path_error = np.sum((x_new * s_new/np.mean(x_new * s_new) - 1)**2)
    assert central_path_error <= 0.25 * mew_new, "central path invariant violated"

    return x_new, y_new, s_new, mew_new

In [233]:
# Iterating until convergence
epsilon = 1e-8
i = 0

A = np.array([
    [3, 3, 3, 0, 0],
    [3, 1, 0, 1, 0],
    [1, 4, 0, 0, 1]
], dtype=float)

b = np.array([4, 3, 4], dtype=float)

c = np.array([-3, -4, 0, 0, 0], dtype=float)
x = np.array([2/5, 8/15, 2/5, 19/15, 22/15], dtype=float)
y = np.array([-4/5, -4/5, -2/3], dtype=float)
s = np.array([37/15, 28/15, 12/5, 4/5, 2/3], dtype=float)
mew = 1

while True:
    x_new, y_new, s, mew = interior_point_method_step_impr(A, b, c, x, y, s, mew, sigma=0.6)
    
    if np.linalg.norm(x_new - x) < epsilon and np.linalg.norm(y_new - y) < epsilon:
        break

    x = x_new
    y= y_new
    i+=1


print("Optimal x:", x)
print(f"y: {y}")
print(f"s: {s}")
print(f"mew: {mew}")
print(f"it took {i} iterations")

Optimal x: [4.44444447e-01 8.88888884e-01 2.28980231e-09 7.77777774e-01
 1.83184184e-08]
y: [-8.88888885e-01 -7.85075078e-09 -3.33333336e-01]
s: [8.24328828e-09 4.12164416e-09 2.66666666e+00 4.71045046e-09
 3.33333335e-01]
mew: 3.6636836884238276e-09
it took 37 iterations


## 10)

In [252]:
# Iterating until convergence
epsilon = 1e-8
i = 0

A = np.array([
    [3, 3, 3, 0, 0],
    [3, 1, 0, 1, 0],
    [1, 4, 0, 0, 1]
], dtype=float)

b = np.array([4, 3, 4], dtype=float)

c = np.array([-3, -4, 0, 0, 0], dtype=float)
x = np.array([2/5, 8/15, 2/5, 19/15, 22/15], dtype=float)
y = np.array([-4/5, -4/5, -2/3], dtype=float)
s = np.array([37/15, 28/15, 12/5, 4/5, 2/3], dtype=float)
mew = 1

while True:
    x_new, y_new, s, mew = interior_point_method_step(A, b, c, x, y, s, mew)
    
    if np.linalg.norm(x_new - x) < epsilon and np.linalg.norm(y_new - y) < epsilon:
        break

    x = x_new
    y= y_new

    if mew < 1e-6:
        for i in range(len(x)):
            if x[i] < s[i]:
                x[i] = 0
            else:
                s[i] = 0
        break

    i+=1


print("Optimal x:", x)
print(f"y: {y}")
print(f"s: {s}")
print(f"mew: {mew}")
print(f"it took {i} iterations")

Optimal x: [0.44444492 0.88888806 0.         0.77777718 0.        ]
y: [-8.88888226e-01 -1.22330363e-06 -3.33333792e-01]
s: [0.         0.         2.66666468 0.         0.33333379]
mew: 9.514576555798441e-07
it took 4 iterations


## 11)

In [283]:
import sympy as sp
import numpy as np


n = 5  
m = 3


x = sp.symbols(f'x1:{n+1}')
s = sp.symbols(f's1:{n+1}')
y = sp.symbols(f'y1:{m+1}')

A = sp.Matrix([
    [3, 3, 3, 0, 0],
    [3, 1, 0, 1, 0],
    [1, 4, 0, 0, 1]
])
b = sp.Matrix([4, 3, 4])
c = sp.Matrix([-3, -4, 0, 0, 0])

eqs = []


for i in range(m):
    eqs.append(sum(A[i,j]*x[j] for j in range(n)) - b[i])


for j in range(n):
    eqs.append(sum(A[i,j]*y[i] for i in range(m)) + s[j] - c[j])


heuristic_constraints = [
    x[2], x[4], 
    s[0], s[1], s[3]         
]
for constr in heuristic_constraints:
    eqs.append(constr)

# Solve symbolically
vars_to_solve = list(x) + list(s) + list(y)
solution = sp.solve(eqs, vars_to_solve, dict=True)

sol = solution[0]  # dict from Symbol -> value

# Convert to decimals
sol_decimal = {k: float(v.evalf()) for k, v in sol.items()}

# Extract back into arrays using the original symbol lists
x_sol = np.array([sol_decimal[xi] for xi in x])
s_sol = np.array([sol_decimal[si] for si in s])
y_sol = np.array([sol_decimal[yi] for yi in y])

# Convert A, b, c to numpy for numeric checks
A_np = np.array(A.tolist(), dtype=float)
b_np = np.array(b.tolist(), dtype=float).reshape(-1)
c_np = np.array(c.tolist(), dtype=float).reshape(-1)

# Feasibility and optimality checks
primal_residual = np.linalg.norm(A_np @ x_sol - b_np)
dual_residual   = np.linalg.norm(A_np.T @ y_sol + s_sol - c_np)
comp_max        = np.max(np.abs(x_sol * s_sol))
nonnegativity   = (np.all(x_sol >= -1e-12) and np.all(s_sol >= -1e-12))

primal_obj = float(c_np @ x_sol)       # c^T x (minimize)
dual_obj   = float(b_np @ y_sol)       # b^T y (maximize dual)

print("=== Exact solution ===")
print("x =", np.round(x_sol, 10))
print("s =", np.round(s_sol, 10))
print("y =", np.round(y_sol, 10))
print()
print("=== Feasibility & optimality diagnostics ===")
print("||Ax - b||              =", primal_residual)
print("||A^T y + s - c||      =", dual_residual)
print("max |x_i * s_i|        =", comp_max)
print("Nonnegativity holds?   =", nonnegativity)
print("Objective gap |c^T x - b^T y| =", abs(primal_obj - dual_obj))




=== Exact solution ===
x = [0.44444444 0.88888889 0.         0.77777778 0.        ]
s = [0.         0.         2.66666667 0.         0.33333333]
y = [-0.88888889  0.         -0.33333333]

=== Feasibility & optimality diagnostics ===
||Ax - b||              = 0.0
||A^T y + s - c||      = 0.0
max |x_i * s_i|        = 0.0
Nonnegativity holds?   = True
Objective gap |c^T x - b^T y| = 0.0


## Comercial solver

In [88]:
from scipy.optimize import linprog

In [98]:
A = np.array([
    [3, 3, 3, 0, 0],
    [3, 1, 0, 1, 0],
    [1, 4, 0, 0, 1]
], dtype=float)

b = np.array([4, 3, 4], dtype=float)

c = np.array([-3, -4, 0, 0, 0], dtype=float)
x = np.array([2/5, 8/15, 2/5, 19/15, 22/15], dtype=float)
y = np.array([-4/5, -4/5, -2/3], dtype=float)
s = np.array([37/15, 28/15, 12/5, 4/5, 2/3], dtype=float)
mew = 1


# simplex 
res_simplex = linprog(c =c,A_eq=A, b_eq = b, method = "simplex")

# interior
res_interior = linprog(c =c,A_eq=A, b_eq = b, method = "interior-point")

# Rsults
print("Simplex:")
print("  x:", res_simplex.x, " fun:", res_simplex.fun, " iterations:", res_simplex.nit)

print("Interior-point:")
print("  x:", res_interior.x, " fun:", res_interior.fun, " iterations:", res_interior.nit)

Simplex:
  x: [0.44444444 0.88888889 0.         0.77777778 0.        ]  fun: -4.888888888888888  iterations: 3
Interior-point:
  x: [4.44444444e-01 8.88888889e-01 2.08281533e-13 7.77777778e-01
 1.89801273e-14]  fun: -4.888888888891923  iterations: 5


C:\Users\marko\AppData\Local\Temp\ipykernel_16992\1015956755.py:17: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res_simplex = linprog(c =c,A_eq=A, b_eq = b, method = "simplex")
C:\Users\marko\AppData\Local\Temp\ipykernel_16992\1015956755.py:20: DeprecationWarning: `method='interior-point'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res_interior = linprog(c =c,A_eq=A, b_eq = b, method = "interior-point")
